In [1]:
import os
os.chdir("..")

from pprint import pprint
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb

from tuxai.dataset import Dataset, Columns
from tuxai.misc import config_logger
from tuxai.report import model_metrics, Report

config_logger()

In [ ]:
report = Report()
df = report.xgboost_model()
df

2022-12-07 11:04:32,109 :: INFO :: tuxai.report :: generating xgboost report


  0%|          | 0/7 [00:00<?, ?it/s]

2022-12-07 11:04:32,122 :: DEBUG :: tuxai.dataset :: [413] loading dataframe
2022-12-07 11:04:39,001 :: DEBUG :: tuxai.models :: computing model signature...
2022-12-07 11:04:39,007 :: DEBUG :: tuxai.models :: model signature : xgboost(413)|92bbd20708bc59ea0196f889ecc6a9d0
2022-12-07 11:04:39,007 :: DEBUG :: tuxai.models :: loading model: xgboost(413)|92bbd20708bc59ea0196f889ecc6a9d0
2022-12-07 11:04:39,063 :: DEBUG :: tuxai.models :: make prediction.
2022-12-07 11:04:39,559 :: DEBUG :: tuxai.dataset :: [413] loading dataframe
2022-12-07 11:04:46,092 :: DEBUG :: tuxai.featureselection :: group collinear options
2022-12-07 11:04:46,094 :: DEBUG :: tuxai.featureselection :: loading correlation matrix from cache
2022-12-07 11:04:51,185 :: INFO :: tuxai.featureselection :: collinear options: 251 groups found, 565 columns removed
2022-12-07 11:04:52,010 :: DEBUG :: tuxai.models :: computing model signature...
2022-12-07 11:04:52,015 :: DEBUG :: tuxai.models :: model signature : xgboost(413)